# Finding Symbols


Finding the ticker symbol, security identifier, the sector, and other metadata is easy if you know where to look.  This guide is intended to introduce some methods for searching, screening, and discovery.

For maximum coverage and functionality, install OpenBB with `[all]` packages.

The examples here will assume that the OpenBB Platform has been installed, the environment is active, and it has been imported into a Python session.  If the installation is fresh, or an extension was just installed, the Python interface will need to be rebuilt.  It will only take a few moments to complete.

In [ ]:
from openbb import obb


The simplest way to find tickers for a company is with a simple fuzzy query.

## Search the SEC

Perform a quick search using the `openbb-sec` provider.

### Find a Company

Use an empty string, `""`, to return the complete list - over 10,000.

In [ ]:
all_companies = obb.equity.search("", provider="sec")

len(all_companies.results)


The SEC sorts this list by market cap.  Applying the `to_df()` method to `all_companies` will show Apple on top

In [ ]:
all_companies.to_df().head(10)


### Find an Institution

Some reporting companies, like invesment trusts and insurance companies, do not have a ticker symbol directly associated with them.  Filers in the US will have a CIK number, used to retrieve documents from the SEC.

In [ ]:
instututions = obb.regulators.sec.institutions_search("Berkshire Hathaway").to_df()
instututions


### Find a Filing

Search for filings by CIK or ticker symbol.

In [ ]:
homestate_filings = obb.equity.fundamental.filings(cik="0000829771", provider="sec")

homestate_filings.to_df().iloc[-1]


Or, search by form type.

In [ ]:
aapl_filings = obb.equity.fundamental.filings("AAPL", type="4", provider="sec")

aapl_filings.to_df().iloc[-1]


## Screen Markets

Screeners provide a targeted search, a tool for comparison and discovery.  Find stocks from around the world with the screener endpoint, and the `openbb-fmp` provider.

### Find Stocks From India

In [ ]:
results = obb.equity.screener(country="IN", provider="fmp").to_df()
display(len(results))
results.head(5).convert_dtypes()


### Search by Sector

In [ ]:
sector_results = obb.equity.screener(country="IN", sector="Financial Services", provider="fmp").to_df()
display(len(sector_results))
sector_results.head(5).convert_dtypes()


### Search by Industry

In [ ]:
industry_results = obb.equity.screener(country="IN", industry="manufacturing").to_df()
display(len(industry_results))
industry_results.head(5).convert_dtypes()


### Search by Exchange

Some countries, like America, have multiple exchanges.  Narrow the search by combining two or more parameters.  The example below finds the companies listed on the American Stock Exchange (AMEX) that are domiciled in China.

In [ ]:
exchange_results = obb.equity.screener(exchange="amex", country="CN").to_df()
display(len(exchange_results))
exchange_results.convert_dtypes()


## Find an Index

List all indices from a source with:

In [ ]:
indices = obb.index.available(provider="yfinance").to_df()
display(len(indices))


Filter the list down by querying the DataFrame.

In [ ]:
indices[indices["name"].str.contains("ASX 200")]


With the `openbb-yfinance` extension, index time series can be  loaded using the ticker symbol or short code.  Non-American indices have a code beginning with the two-letter country code.

In [ ]:
(
    obb.index.market("au_utilities", provider="yfinance").to_df().tail(1)
    == obb.index.market("^AXUJ", provider="yfinance").to_df().tail(1)
)
